In [1]:
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
print("Tensorflow Version: ", tf.__version__)
print(tf.config.list_physical_devices('GPU'))
print("Numbers of GPU Available: ", len(tf.config.list_physical_devices('GPU')))

2025-12-11 15:32:52.499984: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-11 15:32:52.550553: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765438372.562406    6072 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765438372.566585    6072 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765438372.610738    6072 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Tensorflow Version:  2.19.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Numbers of GPU Available:  1


In [2]:
gpus = tf.config.list_physical_devices('GPU')

print("Num GPUs Available: ", len(gpus))
for gpu in gpus:
    print("Name:", gpu.name, "; Type:", gpu.device_type)

Num GPUs Available:  1
Name: /physical_device:GPU:0 ; Type: GPU


In [3]:
tf.config.optimizer.set_jit(False)

## Section 4. Evaluation of Pseudo-RGB Enhancement for RQ2

> **_Can the proposed "Pseudo-RGB" enhancement technique yield statistically significant improvements when applied to the leakage-corrected, rigorous baseline established in RQ1?_**
> 

In [ ]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
import functools
from tensorflow.keras import mixed_precision
from tensorflow.keras.applications.resnet import preprocess_input
from sklearn.utils import class_weight

In [ ]:
CLASS_MAP = {"non-demented": 0, "dementia_very_mild": 1, "dementia_mild": 2, "dementia_moderate": 3}

def apply_pseudo_rgb(image_path, method='duplicate'):
    img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img_gray is None: return np.zeros((224, 224, 3), dtype=np.uint8)
    img_resized = cv2.resize(img_gray, (224, 224))
    
    if method == 'duplicate':
        img_rgb = np.stack([img_resized] * 3, axis=-1)
    elif method == 'jet':
        img_color = cv2.applyColorMap(img_resized, cv2.COLORMAP_JET)
        img_rgb = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)
    elif method == 'viridis':
        img_color = cv2.applyColorMap(img_resized, cv2.COLORMAP_JET) 
        img_rgb = cv2.cvtColor(img_color, cv2.COLOR_BGR2RGB)
    else:
        img_rgb = np.stack([img_resized] * 3, axis=-1)
    return img_rgb

def process_image(file_path_tensor, label_tensor, method):
    file_path = file_path_tensor.numpy().decode('utf-8')
    label = label_tensor.numpy()
    img = apply_pseudo_rgb(file_path, method=method)
    img = img.astype(np.float32)
    img = preprocess_input(img)
    return img, label

def get_class_weights(df):
    y_train = [CLASS_MAP[c] for c in df['class_name'].values]
c    class_weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    return dict(enumerate(class_weights))

def set_shapes(img, label):
    img.set_shape([224, 224, 3])
    label.set_shape([])
    return img, label

def create_dataset(df, method='duplicate', batch_size=32, shuffle=False):
    file_paths = df['file_path'].values
    labels = [CLASS_MAP[c] for c in df['class_name'].values]
    
    ds = tf.data.Dataset.from_tensor_slices((file_paths, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=2000)
    
    loader_func = functools.partial(process_image, method=method)
    
    ds = ds.map(lambda x, y: tf.py_function(loader_func, [x, y], [tf.float32, tf.int32]), 
                num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(set_shapes, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

def build_model(backbone='resnet101', input_shape=(224, 224, 3), num_classes=4):
    inputs = tf.keras.Input(shape=input_shape)
    
    if backbone == 'resnet101':
        base_model = tf.keras.applications.ResNet101(
            include_top=False, weights='imagenet', input_tensor=inputs)
    elif backbone == 'efficientnetb0':
        base_model = tf.keras.applications.EfficientNetB0(
            include_top=False, weights='imagenet', input_tensor=inputs)
    else:
        base_model = tf.keras.applications.ResNet101(
            include_top=False, weights='imagenet', input_tensor=inputs)

    base_model.trainable = False 
    
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    return tf.keras.Model(inputs, outputs, name=f"{backbone}_frozen")

def run_experiment(exp_id, df_train, df_val, backbone='resnet101', method='duplicate', batch_size=32):
    print(f"STARTING {exp_id} | ResNet101 | {method}")
    
    # 計算權重
    class_weights = get_class_weights(df_train)
    print(f"Class Weights: {class_weights}")

    # 建立 Dataset (使用修正後的 process_image)
    train_ds = create_dataset(df_train, method=method, batch_size=batch_size, shuffle=True)
    val_ds = create_dataset(df_val, method=method, batch_size=batch_size, shuffle=False)
    
    model = build_model(weights='imagenet')
    
    # 使用較小的 learning rate，因為我們在微調
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['sparse_categorical_accuracy'] # 不平衡資料建議也看 AUC，但先修好 accuracy 再說
    )
    
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    ]
    
    history = model.fit(
        train_ds, 
        validation_data=val_ds, 
        epochs=15, 
        callbacks=callbacks,
        class_weight=class_weights # 【關鍵修正】：加入權重
    )
    
    return history

In [ ]:
from sklearn.model_selection import GroupShuffleSplit
gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=2)
train_idx, test_idx = next(gss.split(df, groups=df['sid']))
df_train_all = df.iloc[train_idx].copy()

# Internal Split
gss_val = GroupShuffleSplit(n_splits=1, train_size=0.9, random_state=42)
t_idx, v_idx = next(gss_val.split(df_train_all, groups=df_train_all['sid']))
df_train_final = df_train_all.iloc[t_idx].copy()
df_val = df_train_all.iloc[v_idx].copy()

print(f"Data Restored: Train={len(df_train_final)}, Val={len(df_val)}")

Data Restored: Train=54480, Val=5612


In [23]:
acc_a = run_experiment(
    exp_id='EXP_A', 
    df_train=df_train_final, 
    df_val=df_val,
    backbone='resnet101', 
    method='duplicate', 
    batch_size=32
)

STARTING EXP_A | resnet101 | duplicate | BS=32
Epoch 1/15


I0000 00:00:1765440951.165655    9156 service.cc:152] XLA service 0x7fe150002bf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1765440951.165672    9156 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2025-12-11 16:15:51.354807: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1765440952.679091    9156 cuda_dnn.cc:529] Loaded cuDNN version 91002
2025-12-11 16:15:53.385912: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9654', 560 bytes spill stores, 560 bytes spill loads

2025-12-11 16:15:53.422439: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fus

   2/1703 ━━━━━━━━━━━━━━━━━━━━ 2:43 96ms/step - loss: 1.5106 - sparse_categorical_accuracy: 0.0000e+00 

I0000 00:00:1765440956.013394    9156 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1702/1703 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 0.2096 - sparse_categorical_accuracy: 0.9589

2025-12-11 16:18:39.020110: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9654', 560 bytes spill stores, 560 bytes spill loads



1703/1703 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.2096 - sparse_categorical_accuracy: 0.9589

2025-12-11 16:19:00.950431: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3294', 8 bytes spill stores, 8 bytes spill loads

2025-12-11 16:19:01.184406: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_3294', 532 bytes spill stores, 532 bytes spill loads



1703/1703 ━━━━━━━━━━━━━━━━━━━━ 196s 110ms/step - loss: 0.2051 - sparse_categorical_accuracy: 0.9476 - val_loss: 2.1316 - val_sparse_categorical_accuracy: 0.1304
Epoch 2/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 185s 108ms/step - loss: 0.1872 - sparse_categorical_accuracy: 0.9425 - val_loss: 2.1666 - val_sparse_categorical_accuracy: 0.1304
Epoch 3/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 187s 110ms/step - loss: 0.1833 - sparse_categorical_accuracy: 0.9431 - val_loss: 2.1915 - val_sparse_categorical_accuracy: 0.1304
Epoch 4/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 188s 110ms/step - loss: 0.1833 - sparse_categorical_accuracy: 0.9434 - val_loss: 2.1487 - val_sparse_categorical_accuracy: 0.1304
Epoch 5/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 192s 113ms/step - loss: 0.1785 - sparse_categorical_accuracy: 0.9456 - val_loss: 2.2051 - val_sparse_categorical_accuracy: 0.1304
EXP_A Done. Acc: 0.1304


In [24]:
acc_b = run_experiment(
    exp_id='RQ2_Enhanced_Jet', 
    df_train=df_train_final, 
    df_val=df_val, 
    backbone='resnet101', 
    method='jet', 
    batch_size=32
)

STARTING RQ2_Enhanced_Jet | resnet101 | jet | BS=32
Epoch 1/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 201s 114ms/step - loss: 0.2162 - sparse_categorical_accuracy: 0.9425 - val_loss: 2.1213 - val_sparse_categorical_accuracy: 0.1304
Epoch 2/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 191s 112ms/step - loss: 0.1862 - sparse_categorical_accuracy: 0.9434 - val_loss: 2.0378 - val_sparse_categorical_accuracy: 0.1304
Epoch 3/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 188s 111ms/step - loss: 0.1792 - sparse_categorical_accuracy: 0.9447 - val_loss: 2.0411 - val_sparse_categorical_accuracy: 0.1304
Epoch 4/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 188s 110ms/step - loss: 0.1731 - sparse_categorical_accuracy: 0.9463 - val_loss: 2.0094 - val_sparse_categorical_accuracy: 0.1304
Epoch 5/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 187s 110ms/step - loss: 0.1724 - sparse_categorical_accuracy: 0.9463 - val_loss: 1.9599 - val_sparse_categorical_accuracy: 0.1304
Epoch 6/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 187s 109ms/step - loss: 0.1667 - sparse_catego

In [25]:
acc_c = run_experiment(
    exp_id='RQ2_Enhanced_Viridis', 
    df_train=df_train_final, 
    df_val=df_val, 
    backbone='resnet101', 
    method='viridis', 
    batch_size=32
)

STARTING RQ2_Enhanced_Viridis | resnet101 | viridis | BS=32
Epoch 1/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 204s 116ms/step - loss: 0.1819 - sparse_categorical_accuracy: 0.9549 - val_loss: 2.1156 - val_sparse_categorical_accuracy: 0.1304
Epoch 2/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 191s 112ms/step - loss: 0.1783 - sparse_categorical_accuracy: 0.9458 - val_loss: 2.0116 - val_sparse_categorical_accuracy: 0.1304
Epoch 3/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 187s 110ms/step - loss: 0.1753 - sparse_categorical_accuracy: 0.9463 - val_loss: 1.9920 - val_sparse_categorical_accuracy: 0.1304
Epoch 4/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 187s 110ms/step - loss: 0.1710 - sparse_categorical_accuracy: 0.9471 - val_loss: 1.8904 - val_sparse_categorical_accuracy: 0.1304
Epoch 5/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 186s 109ms/step - loss: 0.1664 - sparse_categorical_accuracy: 0.9488 - val_loss: 1.8934 - val_sparse_categorical_accuracy: 0.1304
Epoch 6/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 187s 110ms/step - loss: 0.1634 - spars

In [26]:
# Execution of EXP_D (Architecture Control Baseline)
# Testing the lightest model to confirm VRAM stability before attempting ResNet101 again.
acc_d = run_experiment(
    exp_id='RQ2_Enhanced_efficientnetb0', 
    df_train=df_train_final, 
    df_val=df_val, 
    backbone='efficientnetb0', 
    method='duplicate', 
    batch_size=32
)

STARTING RQ2_Enhanced_efficientnetb0 | efficientnetb0 | duplicate | BS=32
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/15


2025-12-11 18:03:33.628030: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9913', 976 bytes spill stores, 976 bytes spill loads

2025-12-11 18:03:33.657740: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9913', 636 bytes spill stores, 636 bytes spill loads

2025-12-11 18:03:36.608499: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-11 18:03:36.695138: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-11 18:03:37.006090: E external/local_x

1702/1703 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.1524 - sparse_categorical_accuracy: 0.9962

2025-12-11 18:04:54.551624: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_9913', 636 bytes spill stores, 636 bytes spill loads

2025-12-11 18:04:56.879800: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-11 18:04:56.965539: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-11 18:04:57.246621: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-11 18:04:57.334081: E external/local_xla/xla/

1703/1703 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.1525 - sparse_categorical_accuracy: 0.9962

2025-12-11 18:05:10.718198: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2048', 8 bytes spill stores, 8 bytes spill loads

2025-12-11 18:05:10.908705: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2048', 580 bytes spill stores, 580 bytes spill loads

2025-12-11 18:05:12.947747: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-11 18:05:13.032733: E external/local_xla/xla/stream_executor/cuda/cuda_timer.cc:86] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
2025-12-11 18:05:13.303293: E external/local_xla/x

1703/1703 ━━━━━━━━━━━━━━━━━━━━ 107s 56ms/step - loss: 0.1853 - sparse_categorical_accuracy: 0.9539 - val_loss: 2.2648 - val_sparse_categorical_accuracy: 0.1304
Epoch 2/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 81s 47ms/step - loss: 0.1822 - sparse_categorical_accuracy: 0.9437 - val_loss: 2.1984 - val_sparse_categorical_accuracy: 0.1304
Epoch 3/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 81s 47ms/step - loss: 0.1817 - sparse_categorical_accuracy: 0.9432 - val_loss: 2.2564 - val_sparse_categorical_accuracy: 0.1304
Epoch 4/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 81s 47ms/step - loss: 0.1824 - sparse_categorical_accuracy: 0.9435 - val_loss: 2.3184 - val_sparse_categorical_accuracy: 0.1304
Epoch 5/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 80s 47ms/step - loss: 0.1817 - sparse_categorical_accuracy: 0.9432 - val_loss: 2.2476 - val_sparse_categorical_accuracy: 0.1304
Epoch 6/15
1703/1703 ━━━━━━━━━━━━━━━━━━━━ 81s 47ms/step - loss: 0.1806 - sparse_categorical_accuracy: 0.9446 - val_loss: 2.3018 - val_sparse_categorical_accurac